In [1]:
from argparse import ArgumentParser
import os
import random
from matplotlib import pyplot as plt
import torch
from torch import optim
from torch import nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import models
from torchvision.utils import save_image

from data import CityscapesDataset, num_classes, full_to_colour, train_to_full
from model import FeatureResNet, SegResNet

In [2]:
??FeatureResNet

In [3]:
??SegResNet

In [ ]:
# dimension of cityscapes : 2048 x 1024

In [ ]:
# Setup
parser = ArgumentParser(description='Semantic segmentation')
parser.add_argument('--seed', type=int, default=42, help='Random seed')
parser.add_argument('--workers', type=int, default=8, help='Data loader workers')
parser.add_argument('--epochs', type=int, default=100, help='Training epochs')
parser.add_argument('--crop-size', type=int, default=512, help='Training crop size')
parser.add_argument('--lr', type=float, default=5e-5, help='Learning rate')
parser.add_argument('--weight-decay', type=float, default=2e-4, help='Weight decay')
parser.add_argument('--batch-size', type=int, default=4, help='Batch size')
args = parser.parse_args()
random.seed(args.seed)
torch.manual_seed(args.seed)
if not os.path.exists('results'):
    os.makedirs('results')
plt.switch_backend('agg')  # Allow plotting when running remotely

In [12]:
# hyperparmas:
seed = 42
workers = 8
epochs = 100
crop_size = 512
lr = 5e-5
weightdecay = 2e-4
batch_size = 2

In [8]:
torch.manual_seed(seed)
if not os.path.exists('results'):
    os.makedirs('results')

In [9]:
train_dataset = CityscapesDataset(split='train', crop=cropsize, flip=True)

In [10]:
val_dataset = CityscapesDataset(split='val')

In [11]:
type(train_dataset)

data.CityscapesDataset

In [13]:
??DataLoader

In [14]:
# Data
train_dataset = CityscapesDataset(split='train', crop=crop_size, flip=True)
val_dataset = CityscapesDataset(split='val')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=1, num_workers=workers, pin_memory=True)

In [15]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

In [16]:
pretrained_net = FeatureResNet()

In [17]:
??pretrained_net

In [18]:
print(pretrained_net)

FeatureResNet (
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU (inplace)
  (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential (
    (0): BasicBlock (
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock (
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): Batc

In [19]:
pretrained_net.load_state_dict(models.resnet34(pretrained=True).state_dict())

In [26]:
type(pretrained_net)

model.FeatureResNet

In [25]:
type(SegResNet)

type

In [21]:
num_classes

20

In [23]:
net = SegResNet(num_classes, pretrained_net).cuda()

In [24]:
type(net)

model.SegResNet

In [29]:
crit = nn.BCELoss().cuda()

In [31]:
type(crit)

torch.nn.modules.loss.BCELoss

In [27]:
??nn.BCELoss()

In [ ]:
# Training/Testing

# create the network structure for ResNet
pretrained_net = FeatureResNet()

# load the weights
pretrained_net.load_state_dict(models.resnet34(pretrained=True).state_dict())

# plug the ResNet into FCNN framework
net = SegResNet(num_classes, pretrained_net).cuda()

# why use Binary corss-entropy loss
crit = nn.BCELoss().cuda()

# compile the optimizer
optimiser = optim.RMSprop(net.parameters(), lr=args.lr, weight_decay=args.weight_decay)
scores, mean_scores = [], []